In [19]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.auto import tqdm
tqdm.pandas()
from nltk import ngrams

from textblob import TextBlob
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords, movie_reviews

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import enchant
d = enchant.Dict("en_US")
%load_ext autoreload
%autoreload 2
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
cs = set(pd.read_csv("customstop.csv")['Unnamed: 0'].values)

In [81]:
test_tweets = pd.read_csv("test_tweets.csv")
train_tweets = pd.read_csv("train_tweets.csv")
lem = WordNetLemmatizer()
def form_sentence(tweet):
    tbt = TextBlob(tweet).words
    w = ' '.join(tbt).split(" ")
    for i in w:
        if len(i) == 0:
            w.remove(i)
    checkedt = [word.lower() for word in w if d.check(word)]
    nostopt = [word for word in checkedt if word not in cs]
    lemt = [lem.lemmatize(word,'v') for word in nostopt]
    return lemt

In [82]:
import copy

## QUALITY

In [161]:
filt = pd.read_csv("filtered data set.csv")

In [162]:
filt["Commentstf"] = filt["Comments"].apply(lambda x: len(x.split(" ")) > 10)
filt = filt[filt["Commentstf"] == True].drop(["Commentstf"],axis = 1)

In [163]:
coursereaeval = pd.read_csv("reviews.csv")
coursereaeval['old'] = coursereaeval["Review"]
coursereaeval["tweet"] = coursereaeval["old"].progress_apply(form_sentence)
filt = pd.read_csv("filtered data set.csv")


In [164]:
filt["tweet"] = filt.Comments.progress_apply(form_sentence)

In [165]:
al = pd.read_csv("keyWords.csv").columns[10]
al

'Quality'

In [166]:
huh = pd.read_csv("keyWords.csv")[al]
topw = set(list(huh.values))
filt['widf'] = filt['tweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in topw})


In [167]:
finaldf = pd.concat([coursereaeval[['tweet','Label']]])
finaldf['widf'] = coursereaeval['tweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in topw})

In [168]:
forc = finaldf[['widf','Label']]
forctuple = list(forc.itertuples(index=False, name=None))

In [169]:
import time

In [170]:
start = time.time()
train_set, test_set = forctuple[100:], forctuple[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(time.time() - start)

9.535282850265503


In [171]:
print(nltk.classify.accuracy(classifier, test_set))

0.7


In [172]:
filt['classifiedlearning'] = filt['widf'].progress_apply(lambda wv: classifier.classify(wv))

In [173]:
classifier.show_most_informative_features(100)

Most Informative Features
                response = True                1 : 5      =     19.2 : 1.0
                      ta = True                2 : 5      =     15.7 : 1.0
                 tedious = True                2 : 5      =     14.7 : 1.0
                    chat = True                1 : 5      =     14.1 : 1.0
                  assign = True                2 : 5      =     13.1 : 1.0
                 outside = True                1 : 5      =     12.8 : 1.0
                    talk = True                2 : 5      =      9.1 : 1.0
                    slow = True                2 : 5      =      8.8 : 1.0
                question = True                2 : 5      =      8.8 : 1.0
                   final = True                2 : 5      =      8.5 : 1.0
              assignment = True                2 : 5      =      7.6 : 1.0
              discussion = True                1 : 5      =      7.5 : 1.0
                     lab = True                2 : 1      =      6.9 : 1.0

In [175]:
# filt[["Course Key","Comments","tweet","classified"]].to_csv("analysed cc - first pass")

In [178]:
for i in range(1,6):
    print(len(filt[filt['classifiedlearning'] == i]),f"{i}")

329 1
2460 2
656 3
2446 4
12887 5


In [180]:
pd.set_option('max_colwidth', 600)
filt[filt['classifiedlearning'] == 5][['Comments','classifiedlearning']]

,Comments,classifiedlearning
0,3.5 out of 5 stars,5
2,"a lot of reading on your own part, but interesting overall",5
5,Dense material and too much focus on minute details. Good professors though.,5
6,"Discussion sections good, but perhaps overused (too many weeks out of the semester). ""What's new"" a nice feature.",5
7,Dr. Gustin is so funny. I loved his lectures.,5
9,"Gustin is an amazing teacher. His lectures are insightful and interesting enough to keep me focused. He explains the topics with several different methods (such as writing on the chalkboard, handouts, verbal communication...) and this helps me understand. Gustin also has something called ""The What's New Feature"" during the middle of class that gives everyone a small five minute break. The chance to take a breath is much appreciated in the morning.\r\n\r\nThis teacher really makes me feel secure in the material. He obviously cares about the quality of his teaching and the education that is ...",5
10,Hard but good and excellent teaching.,5
11,I didn't enjoy the subject matter as much as I thought I would.,5
13,"I felt that the tests often demanded a deeper understanding of the material than was attainable through attending lectures, recitations sessions, etc.",5
14,I found the course very interesting and enjoyed the material.,5


## HAPPINESS

In [184]:
huh = pd.read_csv("keyWords.csv")[["Happy","Unhappy"]]
topw = set(list(huh.Happy.values) + list(huh.Unhappy.values))
filt['widf'] = filt['tweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in topw})


In [185]:
finaldf = pd.concat([coursereaeval[['tweet','Label']]])
finaldf['widf'] = coursereaeval['tweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in topw})

In [186]:
forc = finaldf[['widf','Label']]
forctuple = list(forc.itertuples(index=False, name=None))

In [187]:
import time

In [188]:
start = time.time()
train_set, test_set = forctuple[100:], forctuple[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(time.time() - start)

808.2470939159393


In [189]:
print(nltk.classify.accuracy(classifier, test_set))

0.73


In [191]:
# filt['classifiedhappy'] = filt['widf'].progress_apply(lambda wv: classifier.classify(wv))
filt["classifiedhappy"] = classifier.classify_many(filt['widf'].values)

In [192]:
classifier.show_most_informative_features(100)

Most Informative Features
                   worst = True                1 : 5      =    392.2 : 1.0
                  refund = True                1 : 5      =    391.2 : 1.0
                  poorly = True                1 : 5      =    352.7 : 1.0
             meaningless = True                1 : 5      =    181.7 : 1.0
                   worse = True                1 : 5      =    175.1 : 1.0
            inconsistent = True                1 : 5      =    160.3 : 1.0
                   buggy = True                1 : 5      =    160.3 : 1.0
                 garbage = True                1 : 5      =    147.5 : 1.0
              ridiculous = True                1 : 5      =    146.1 : 1.0
              inaccurate = True                1 : 5      =    138.9 : 1.0
         inconsistencies = True                1 : 5      =    138.9 : 1.0
                 poorest = True                1 : 5      =    138.9 : 1.0
                    poor = True                1 : 5      =    138.9 : 1.0

In [193]:
# filt[["Course Key","Comments","tweet","classified"]].to_csv("analysed cc - first pass")

In [194]:
for i in range(1,6):
    print(len(filt[filt['classifiedhappy'] == i]),f"{i}")

1303 1
1081 2
1430 3
3544 4
11420 5


In [197]:
pd.set_option('max_colwidth', 600)
filt[filt['classifiedhappy'] == 5][['Comments','classifiedhappy']]

,Comments,classifiedhappy
0,3.5 out of 5 stars,5
2,"a lot of reading on your own part, but interesting overall",5
6,"Discussion sections good, but perhaps overused (too many weeks out of the semester). ""What's new"" a nice feature.",5
7,Dr. Gustin is so funny. I loved his lectures.,5
9,"Gustin is an amazing teacher. His lectures are insightful and interesting enough to keep me focused. He explains the topics with several different methods (such as writing on the chalkboard, handouts, verbal communication...) and this helps me understand. Gustin also has something called ""The What's New Feature"" during the middle of class that gives everyone a small five minute break. The chance to take a breath is much appreciated in the morning.\r\n\r\nThis teacher really makes me feel secure in the material. He obviously cares about the quality of his teaching and the education that is ...",5
10,Hard but good and excellent teaching.,5
11,I didn't enjoy the subject matter as much as I thought I would.,5
12,I enjoyed the material covered in this course but later heard that the syllabus was rearranged from previous years. I would have liked to spent less time on genetics and more on animal systems and the human body as an intro course.,5
13,"I felt that the tests often demanded a deeper understanding of the material than was attainable through attending lectures, recitations sessions, etc.",5
14,I found the course very interesting and enjoyed the material.,5


In [199]:
filt[["Course Key","Comments","tweet","classifiedhappy",'classifiedlearning']].to_csv("analysed cc - first pass.csv")

# BIGRAM ATTEMPT

In [6]:
cs = set(pd.read_csv("customstop.csv")['Unnamed: 0'].values)

In [7]:
lem = WordNetLemmatizer()
def form_sentence_bigram(tweet):
    tbt = TextBlob(tweet).words
    w = ' '.join(tbt).split(" ")
    for i in w:
        if len(i) == 0:
            w.remove(i)
    checkedt = [word.lower() for word in w if d.check(word)]
    nostopt = [word for word in checkedt if word not in cs]
    lemt = [lem.lemmatize(word,'v') for word in nostopt]
    ngram_vocab = ngrams(lemt, 2)
    return list(ngram_vocab)

In [8]:
coursereaeval['bitweet'] = coursereaeval['Review'].progress_apply(form_sentence_bigram)

In [12]:
cutcourseeval = coursereaeval.head(10000)

In [13]:
huh = pd.read_csv("keyWords.csv")[["Happy","Unhappy"]]
topw = list(huh["Happy"].values) + list(huh["Unhappy"].values)
bitopw = list(ngrams(topw, 2))

In [14]:
bifinaldf = pd.concat([cutcourseeval[['bitweet','Label']]])
bifinaldf['widf'] = cutcourseeval['bitweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in bitopw})

In [27]:
bifinaldf['bitweet'] = cutcourseeval['bitweet']

In [25]:
bifinaldf['widf'].iloc[0]

{('a+', 'abound'): False,
 ('abound', 'abounds'): False,
 ('abounds', 'abundance'): False,
 ('abundance', 'abundant'): False,
 ('abundant', 'accessable'): False,
 ('accessable', 'accessible'): False,
 ('accessible', 'acclaim'): False,
 ('acclaim', 'acclaimed'): False,
 ('acclaimed', 'acclamation'): False,
 ('acclamation', 'accolade'): False,
 ('accolade', 'accolades'): False,
 ('accolades', 'accommodative'): False,
 ('accommodative', 'accomodative'): False,
 ('accomodative', 'accomplish'): False,
 ('accomplish', 'accomplished'): False,
 ('accomplished', 'accomplishment'): False,
 ('accomplishment', 'accomplishments'): False,
 ('accomplishments', 'accurate'): False,
 ('accurate', 'accurately'): False,
 ('accurately', 'achievable'): False,
 ('achievable', 'achievement'): False,
 ('achievement', 'achievements'): False,
 ('achievements', 'achievible'): False,
 ('achievible', 'acumen'): False,
 ('acumen', 'adaptable'): False,
 ('adaptable', 'adaptive'): False,
 ('adaptive', 'adequate'): Fal

In [16]:
filt["bitweet"] = filt.Comments.progress_apply(form_sentence_bigram)
filt['biwidf'] = filt['bitweet'].progress_apply(lambda kw: {w:(True if w in kw else False) for w in bitopw})


In [ ]:
## need to create a new list of feature words

In [40]:
v = filt['bitweet'].values[0]

In [46]:
pd.Series(Counter([f"{a[0]}" + " " + f"{a[1]}" for a in v])).sort_values(ascending = False)

the test                   1
different people           1
the concepts               1
of the                     1
some of                    1
it some                    1
teach it                   1
where teach                1
personalities where        1
different personalities    1
very different             1
two very                   1
with two                   1
people with                1
two different              1
like the                   1
be two                     1
there be                   1
when there                 1
especially when            1
bore especially            1
be bore                    1
it be                      1
sometimes it               1
and sometimes              1
read and                   1
of read                    1
lot of                     1
concepts be                1
be interest                1
interest some              1
some be                    1
i like                     1
concept i                  1
this concept  

In [28]:
biforc = bifinaldf[['bitweet','Label']]
biforctuple = list(biforc.itertuples(index=False, name=None))

In [29]:
start = time.time()
bitrain_set, bitest_set = biforctuple[100:], biforctuple[:100]
biclassifier = nltk.DecisionTreeClassifier.train(bitrain_set)
print(time.time() - start)

AttributeError: 'list' object has no attribute 'get'

In [21]:
print(nltk.classify.accuracy(biclassifier, bitest_set))

0.7


In [22]:
# filt['biclassified'] = filt['biwidf'].progress_apply(lambda wv: biclassifier.classify(wv))
filt["biclassified"] = biclassifier.classify_many(filt['biwidf'].values)

In [23]:
for i in range(1,6):
    print(len(filt[filt['biclassified'] == i]),f"{i}")

0 1
0 2
0 3
0 4
16695 5


# OUTDATED BUT TOO SCARED TO DELETE

In [147]:
finaldf = pd.concat([movreviews[['tweet','label']]])
# finaldf = train_tweets[['tweet','label']]

def text_processing(inp):
    return Counter(inp)

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_processing)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

msg_train, msg_test, label_train, label_test = train_test_split(finaldf['tweet'], finaldf['label'], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))
filt['label'] = pipeline.predict(filt['tweet'])

              precision    recall  f1-score   support

           0       0.76      0.83      0.79       190
           1       0.83      0.76      0.80       210

    accuracy                           0.80       400
   macro avg       0.80      0.80      0.79       400
weighted avg       0.80      0.80      0.80       400

[[158  32]
 [ 50 160]]
0.795


In [148]:
len(filt[filt['label'] == 1])

10839

In [149]:
len(filt[filt['label'] == 0])

5856

In [151]:
filt[filt['label'] == 1]

,Unnamed: 0,Course Key,Term Code,Comments,tweet,label
2,2208,200710-11026,200710,"a lot of reading on your own part, but interesting overall","[lot, read, part, interest, overall]",1
3,2209,200710-11026,200710,"BIOS 201 is a great class,not only because of the material it covers, but because the class has a very relaxed atmosphere. The discussion sessions are helpful, but sometimes the preparation forms were so tedious, it didn't foster enough conversation. However it does provide a structured way for ...","[bios, 201, great, class, material, cover, class, relax, atmosphere, discussion, sessions, helpful, sometimes, preparation, form, tedious, foster, enough, conversation, however, provide, structure, way, students, learn, experience, material, interest, professors, usually, good, job, inspire, int...",1
4,2210,200710-11026,200710,"Bios 201 was an organized, well taught class. At times a lecture method of teaching made it difficult to follow in such a large class, but mostly this was not a problem. Also, discussion sessions helped us understand certain topics in greater detail and allowed us to discuss them in a more perso...","[201, organize, well, teach, class, time, lecture, method, teach, make, difficult, follow, large, class, mostly, problem, also, discussion, sessions, help, us, understand, certain, topics, greater, detail, allow, us, discuss, personal, atmosphere, ta, also, helpful, discussion, sessions, explain...",1
8,2214,200710-11026,200710,Extremely difficult course. I would prepare for tests by studying for at least 20 hourse in the week leading up and my graded does not \r\nreflect at all how hard I worked in this course. The hardest course next to Chemistry that I have ever taken in my life. I do not appreciate\r\nthe fact t...,"[extremely, difficult, course, would, prepare, test, study, least, 20, week, lead, grade, reflect, hard, work, course, hardest, course, next, chemistry, ever, take, life, appreciate, fact, hard, work, probably, go, unnoticed]",1
9,2215,200710-11026,200710,"Gustin is an amazing teacher. His lectures are insightful and interesting enough to keep me focused. He explains the topics with several different methods (such as writing on the chalkboard, handouts, verbal communication...) and this helps me understand. Gustin also has something called ""The Wh...","[amaze, teacher, lecture, insightful, interest, enough, keep, focus, explain, topics, several, different, methods, write, chalkboard, handouts, verbal, communication, help, understand, also, something, call, new, feature, middle, class, give, everyone, small, five, minute, break, chance, take, b...",1
11,2217,200710-11026,200710,I didn't enjoy the subject matter as much as I thought I would.,"[enjoy, subject, matter, much, think, would]",1
12,2218,200710-11026,200710,I enjoyed the material covered in this course but later heard that the syllabus was rearranged from previous years. I would have liked to spent less time on genetics and more on animal systems and the human body as an intro course.,"[enjoy, material, cover, course, later, hear, syllabus, rearrange, previous, years, would, like, spend, less, time, genetics, animal, systems, human, body, intro, course]",1
13,2219,200710-11026,200710,"I felt that the tests often demanded a deeper understanding of the material than was attainable through attending lectures, recitations sessions, etc.","[felt, test, often, demand, deeper, understand, material, attainable, attend, lecture, recitations, sessions, etc]",1
15,2221,200710-11026,200710,I loved the discussion sessions. Having to prepare for discussion and then disseminate the topics really helped me learn.,"[love, discussion, sessions, prepare, discussion, disseminate, topics, really, help, learn]",1
17,2223,200710-11026,200710,"I really liked the discussion sessions. They were very helpful in explaining tricky concepts, and always applied to events happening in every day life (such as going in depth on how cancer starts). It also provid

In [152]:
filt[filt['label'] == 0]

,Unnamed: 0,Course Key,Term Code,Comments,tweet,label
1,2207,200710-11026,200710,"A lot of reading and sometimes it was boring. Especially when there was two different people with two very different personalities where teaching it. some of the concepts were interesting, some were incredibly boring and seemed pointless. I would often say to myself that ""I dont need to know thi...","[lot, read, sometimes, bore, especially, two, different, people, two, different, personalities, teach, concepts, interest, incredibly, bore, seem, pointless, would, often, say, need, know, stuff, care, concept, like, test]",0
5,2211,200710-11026,200710,Dense material and too much focus on minute details. Good professors though.,"[dense, material, much, focus, minute, detail, good, professors, though]",0
6,2212,200710-11026,200710,"Discussion sections good, but perhaps overused (too many weeks out of the semester). ""What's new"" a nice feature.","[discussion, section, good, perhaps, overuse, many, weeks, semester, new, nice, feature]",0
16,2222,200710-11026,200710,I really enjoyed BIOS 201. The professor really has the abilty to make the material exciting.,"[really, enjoy, bios, 201, professor, really, make, material, excite]",0
18,2224,200710-11026,200710,"I really liked this class. I learned a lot from it. It is tough- especially the tests because we are expected to know SO much for the tests, but we retain the information pretty well. The discussion sessions are fun, though the preparation forms require a lot of work. I was a little unsure o...","[really, like, class, learn, lot, tough, especially, test, expect, know, much, test, retain, information, pretty, well, discussion, sessions, fun, though, preparation, form, require, lot, work, little, unsure, exactly, expect, us, form, often, little, confuse, grade, ta, seem, take, random, poin...",0
21,2227,200710-11026,200710,"I thought it was very easy, but some of my friends disagree. It might just be because I really like this material, and have learned much of it before, but I wish that it was a little more challenging. The tests were almost fun, I would have like some homework, if nothing else as a distraction ...","[think, easy, friends, disagree, might, really, like, material, learn, much, wish, little, challenge, test, almost, fun, would, like, homework, nothing, else, distraction, homework, class]",0
22,2228,200710-11026,200710,"I thought the course moved very quickly, and I hated the change in teachers arounf the third quarter of the semester.","[think, course, move, quickly, hat, change, teachers, third, quarter, semester]",0
24,2230,200710-11026,200710,"I'm not generally a fan of courses that are graded by basically tests only, but I did learn a bit. The discussion sessions were annoying.","[generally, fan, course, grade, basically, test, learn, bite, discussion, sessions, annoy]",0
28,2234,200710-11026,200710,Interesting and broad-ranging course. Only complaint is course organization - why is molecular biology stuck in the middle of cell signalling and plant structure? Maybe it should be a separate course.,"[interest, course, complaint, course, organization, molecular, biology, stick, middle, cell, plant, structure, maybe, separate, course]",0
29,2235,200710-11026,200710,"It covers an unbelievable amount of new materials, but as long as you keep up with the reading and go to the lectures, it only gets slightly overwhelming. The organization was good, we learned everything in order, there wasn't much back-tracking. The discussion sessions helped a lot to increase ...","[cover, unbelievable, amount, new, materials, long, keep, read, go, lecture, get, slightly, overwhelm, organization, good, learn, everything, order, much, discussion, sessions, help, lot, increase, understand, material, although, word, question, vague, confuse, time, many, us, misinterpret, want...",0


In [218]:
filt.head()

,Unnamed: 0,Course Key,Term Code,Comments,tweet,widf,classified
1,2207,200710-11026,200710,"A lot of reading and sometimes it was boring. Especially when there was two different people with two very different personalities where teaching it. some of the concepts were interesting, some were incredibly boring and seemed pointless. I would often say to myself that ""I dont need to know thi...","[lot, read, sometimes, bore, especially, two, different, people, two, different, personalities, teach, concepts, interest, incredibly, bore, seem, pointless, would, often, say, need, know, stuff, care, concept, like, test]","{'a+': False, 'abound': False, 'abounds': False, 'abundance': False, 'abundant': False, 'accessable': False, 'accessible': False, 'acclaim': False, 'acclaimed': False, 'acclamation': False, 'accolade': False, 'accolades': False, 'accommodative': False, 'accomodative': False, 'accomplish': False,...",1
2,2208,200710-11026,200710,"a lot of reading on your own part, but interesting overall","[lot, read, part, interest, overall]","{'a+': False, 'abound': False, 'abounds': False, 'abundance': False, 'abundant': False, 'accessable': False, 'accessible': False, 'acclaim': False, 'acclaimed': False, 'acclamation': False, 'accolade': False, 'accolades': False, 'accommodative': False, 'accomodative': False, 'accomplish': False,...",5
3,2209,200710-11026,200710,"BIOS 201 is a great class,not only because of the material it covers, but because the class has a very relaxed atmosphere. The discussion sessions are helpful, but sometimes the preparation forms were so tedious, it didn't foster enough conversation. However it does provide a structured way for ...","[bios, 201, great, class, material, cover, class, relax, atmosphere, discussion, sessions, helpful, sometimes, preparation, form, tedious, foster, enough, conversation, however, provide, structure, way, students, learn, experience, material, interest, professors, usually, good, job, inspire, int...","{'a+': False, 'abound': False, 'abounds': False, 'abundance': False, 'abundant': False, 'accessable': False, 'accessible': False, 'acclaim': False, 'acclaimed': False, 'acclamation': False, 'accolade': False, 'accolades': False, 'accommodative': False, 'accomodative': False, 'accomplish': False,...",4
4,2210,200710-11026,200710,"Bios 201 was an organized, well taught class. At times a lecture method of teaching made it difficult to follow in such a large class, but mostly this was not a problem. Also, discussion sessions helped us understand certain topics in greater detail and allowed us to discuss them in a more perso...","[201, organize, well, teach, class, time, lecture, method, teach, make, difficult, follow, large, class, mostly, problem, also, discussion, sessions, help, us, understand, certain, topics, greater, detail, allow, us, discuss, personal, atmosphere, ta, also, helpful, discussion, sessions, explain...","{'a+': False, 'abound': False, 'abounds': False, 'abundance': False, 'abundant': False, 'accessable': False, 'accessible': False, 'acclaim': False, 'acclaimed': False, 'acclamation': False, 'accolade': False, 'accolades': False, 'accommodative': False, 'accomodative': False, 'accomplish': False,...",5
5,2211,200710-11026,200710,Dense material and too much focus on minute details. Good professors though.,"[dense, material, much, focus, minute, detail, good, professors, though]","{'a+': False, 'abound': False, 'abounds': False, 'abundance': False, 'abundant': False, 'accessable': False, 'accessible': False, 'acclaim': False, 'acclaimed': False, 'acclamation': False, 'accolade': False, 'accolades': False, 'accommodative': False, 'accomodative': False, 'accomplish': False,...",4


In [221]:
filt[['classified','tweet']].apply(lambda a: a[0].append(a[1]))

KeyError: (0, 'occurred at index classified')